Contains the base ideas for our linear program code

In [1]:
from pyscipopt import Model, quicksum
from simulator import combat_sim
from simulator import test_units
from simulator import get_health
from simulator import get_damage

This line here will eventually be swapped out with a function call to read our database of unit stats

In [2]:
Units = test_units()

Viability function copied directly from earlier, separate snippet

In [3]:
def viability(enemy_comp, test_comp, test_type='units'):
    """
    Input is the enemy army composition, the army composition that
    we are testing, and the type of viablity test we are running.
    test_type is a string, either "units", "health", or "damage"
    Runs simulation 200 times. If the test army has at
    least 10% of its test type remaining 90% of the time, return 1
    Otherwise return 0
    """
    test_limit = 0
    if test_type == 'units':
        unit_count = 0
        for unit in test_comp:
            unit_count += test_comp[unit]
        test_limit = 0.1 * unit_count
    
    if test_type == 'health':
        army_health = 0
        for unit in test_comp:
            unit_hp = Units[unit]['hp']
            army_health += unit_hp * test_comp[unit]
        test_limit = 0.1 * army_health
    if test_type == 'damage':
        army_dmg = 0
        for unit in test_comp:
            unit_dmg = Units[unit]['dps']
            army_dmg += unit_dmg * test_comp[unit]
        test_limit = 0.1 * army_dmg
    
    wins = 0
    sim_count = 200
    sims = sim_count
    while sims > 0:
        test_army = combat_sim(enemy_comp, test_comp)
        if (test_type == 'units') and (len(test_army) > test_limit):
            wins += 1
        elif (test_type == 'health') and (get_health(test_army) > test_limit):
            wins += 1
        elif (test_type == 'damage') and (get_damage(test_army) > test_limit):
            wins += 1
        sims -= 1
    if (wins / sim_count) > 0.9:
        return 1
    else:
        return 0

First build of our main code only deals with one linear program (optimize for resource cost). Later builds should include a parameter so that we can reuse most of the code for other optimizations such as total build time.

Right now, resource cost is calculated much in the same way the army value stat is calculated in-game; the resource cost of a unit is the sum of the gas and mineral cost of that unit. How much a player actually values gas vs minerals usually depends on how much they currently have available and how much they are producing. A skilled player might also have an idea of what sort of units or research they want to be building.

Recall for the linear program, objective function is to minimize
$$\sum_{name} cost[name] \cdot count[name]$$
with the two constraints being stay below the supply cap
$$\sum_{name} count[name] \cdot supply[name] \leq 200$$
and make sure that army comp is viable
$$viable_{army} \geq 1$$

The task at hand is to figure out how to deal with creating these variables in our linear program. The viable variable is per army composition, and so to initialize it we would need to preconstruct and itterate through all possible army comps with supply $\leq$ 200, then assign each of those comps a viable variable.

Basically, the issue is that the viable variable deals with army compositions as a whole, not individual units

In [ ]:
def init_army_comps(race='Terran'):
    """
    Input is the race we wish to build army comps for
    Creates a list of all possible army compositions with
    total supply <= 200
    Returns that list of army compositions
    """
    

Main function for our linear program will be below

In [ ]:
def find_optimal_army(enemy_comp, test_type='cost', race='Terran', viab='units'):
    """
    Input is the army composition we are trying to optimize against
    test_type is a string, either "cost" or "time", determines what we
    are optimizing for. race is a string, either "Terran", "Protoss", or "Zerg",
    that we are creating our test army from.
    viab is a string, either 'units', 'health', or 'damage' that determines
    our viability function
    Finds an army composition of the given race that is the most efficent in terms of test_type
    Returns that optimal army composition
    """
    army_model = Model("StarCraftII Army")
    
    
    